# Poisson equation solution using FEeniCSx/DOLFINx
## Poisson equation variational formulation
Poisson equation is defined as follows:
$$ -\nabla^2 u(\mathbf{x}) = f(\mathbf{x}), \ \mathbf{x} \in \Omega\\
u(\mathbf{x}) = u_D (\mathbf{x}), \ \mathbf{x} \in \partial \Omega,$$

and is the generalized form of the Laplace equation with a non-homogenous right hand side. The variational form of the Poisson equation can be generated simply by multiplying both sides with a \emph{test} function, $v(\mathbf{x})$, such that:
$$ -\int_\Omega \nabla^2 u \ v~\mathrm{d}x = - \int_\Omega f \ v~\mathrm{d}x.$$
Applying integration by part on this expressions yields,
$$ -\int_\Omega (\nabla^2 u)v~\mathrm{d}x = 
\int_\Omega\nabla u\cdot\nabla v~\mathrm{d}x - 
\int_{\partial\Omega}\frac{\partial u}{\partial n}v~\mathrm{d}s,$$
and $v(\mathbf{x})$ is chosen such that it vanishes where $u(\mathbf{x})$ is known. Therefore, it becomes quite simply,
$$ -\int_\Omega (\nabla^2 u)v~\mathrm{d}x = 
\int_\Omega\nabla u\cdot\nabla v~\mathrm{d}x.$$
Plugging this simplification back into the original expression gives,
$$\int_\Omega \nabla u \cdot \nabla v \mathrm{d} x = \int_\Omega f\ v~\mathrm{d} x.$$
The above can be cast in the canonical form of variational problem as
$$ a(u,v) = L(v), ~\forall v \in V,$$
where $a(u,v)$ is known as the bilinear form and $L(v)$ is known as the linear problem

## Implementation details
* `MPI.COMM_SELF`: Each processor node gets one instance of mesh. Useful for running simulations with different parameters. 
* `MPI.COMM_WORLD`: Single mesh distributed across different nodes
* `dolfinx.fem`: This module contains functions spaces, functions, and solvers for FEM. This can translate variational problems into linear/non-linear systems that can be solved by `PETSc`.
* `dolfinx.mesh`: Generates meshs/solution domains.
* `ufl`: Unified Form Language that can be used define the variational problems.

In [1]:
from mpi4py import MPI
import dolfinx as fe
import numpy
import ufl
from petsc4py import PETSc

In [2]:
# Generate mesh
solution_domain = fe.mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, fe.mesh.CellType.quadrilateral)
V_function_space = fe.fem.FunctionSpace(solution_domain, ("CG", 1))

In [3]:
# Defining boundary condition
u_boundary = fe.fem.Function(V_function_space)
u_boundary.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)

In [4]:
# Create facet to cell connectivity required to determine boundary facets
tdim = solution_domain.topology.dim
fdim = tdim - 1
solution_domain.topology.create_connectivity(fdim, tdim)
boundary_facets = fe.mesh.exterior_facet_indices(solution_domain.topology)

In [5]:
# Define boundary degrees of freedom
boundary_dofs = fe.fem.locate_dofs_topological(V_function_space, fdim, boundary_facets)
bc = fe.fem.dirichletbc(u_boundary, boundary_dofs)

In [6]:
u = ufl.TrialFunction(V_function_space)
v = ufl.TestFunction(V_function_space)

In [7]:
# The scalar $f$ is defined as a constant
f_num = - 6
f = fe.fem.Constant(solution_domain, PETSc.ScalarType(f_num))

In [8]:
# Variational problem
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

In [22]:
# Solution
problem = fe.fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [52]:
uh.vector.array

array([1.      , 1.03125 , 1.015625, 1.046875, 1.125   , 1.140625,
       1.0625  , 1.09375 , 1.28125 , 1.296875, 1.1875  , 1.140625,
       1.171875, 1.5     , 1.515625, 1.34375 , 1.265625, 1.25    ,
       1.28125 , 1.78125 , 1.796875, 1.5625  , 1.421875, 1.375   ,
       1.390625, 1.421875, 2.125   , 2.140625, 1.84375 , 1.640625,
       1.53125 , 1.515625, 1.5625  , 1.59375 , 2.53125 , 2.546875,
       2.1875  , 1.921875, 1.75    , 1.671875, 1.6875  , 1.765625,
       1.796875, 3.      , 3.015625, 2.59375 , 2.265625, 2.03125 ,
       1.890625, 1.84375 , 1.890625, 2.      , 2.03125 , 3.0625  ,
       2.671875, 2.375   , 2.171875, 2.0625  , 2.046875, 2.125   ,
       3.140625, 2.78125 , 2.515625, 2.34375 , 2.265625, 2.28125 ,
       3.25    , 2.921875, 2.6875  , 2.546875, 2.5     , 3.390625,
       3.09375 , 2.890625, 2.78125 , 3.5625  , 3.296875, 3.125   ,
       3.765625, 3.53125 , 4.      ])